# Assignment Chapter 3 - COMPUTER VISION [Case #3]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Dataset: Any YouTube videos
* Libraries: PyTorch, Numpy, OpenCV2
* Objective: Real-time Object Detection using CNN-based Pre-trained Models

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengimplementasikan model PyTorch untuk mendeteksi objek secara *real-time*.

### Install additional library

In [ ]:
!pip install cap-from-youtube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.5 MB/s eta 0:00:00


### Import Libraries

In [ ]:
import torch
import numpy as np
import cv2

from cap_from_youtube import cap_from_youtube
from time import time
from datetime import datetime as dt

### User-defined Class

In [ ]:
class ObjectDetection:
    def __init__(self, url, out_file="{}_video.avi".format(dt.now().strftime("%Y%m%d_%H%M%S"))):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """

        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """

        return cap_from_youtube(self._URL)

    def load_model(self):
        """
        Loads the model from pytorch hub.
        :return: Trained Pytorch model.
        """

        # TODO: Panggil model ultralytics/yolov5
        # Lihat caranya di https://pytorch.org/hub/ultralytics_yolov5/#load-from-pytorch-hub
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """

        self.model.to(self.device)

        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].cpu().numpy(), results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """

        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """

        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """

        player = self.get_video_from_url()
        assert player.isOpened()

        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))

        for i in range(1, 300):
            start_time = time()
            ret, frame = player.read()

            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()

            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)

### IMPORTANT: Activate your GPU

- Di Google Colab, klik **Runtime > Change runtime time**, lalu silakan pilih **T4 GPU**.

### Start Object Detection

In [ ]:
if __name__ == "__main__":
    # Pastikan CUDA enable: TRUE
    print("CUDA enable: {}".format(torch.cuda.is_available()))

    # TODO: Isi parameter dengan URL YouTube yang tersedia (secara bergantian):
    # 1. Crowded place: https://www.youtube.com/watch?v=dwD1n7N7EAg
    # 2. Solar system: https://www.youtube.com/watch?v=g2KmtA97HxY
    # 3. Road traffic: https://www.youtube.com/watch?v=wqctLW0Hb_0

    run_model = ObjectDetection(url="https://www.youtube.com/watch?v=dwD1n7N7EAg")
    run_model()

CUDA enable: True


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-23 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


[youtube] Extracting URL: https://www.youtube.com/watch?v=dwD1n7N7EAg
[youtube] dwD1n7N7EAg: Downloading webpage
[youtube] dwD1n7N7EAg: Downloading ios player API JSON
[youtube] dwD1n7N7EAg: Downloading mweb player API JSON
[youtube] dwD1n7N7EAg: Downloading m3u8 information


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 3.6101083032490973
Frames Per Second : 22.72727272727273
Frames Per Second : 14.084507042253522

...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 37.03703703703704
Frames Per Second : 35.714285714285715
Frames Per Second : 45.45454545454546
Frames Per Second : 41.666666666666664


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 30.3030303030303
Frames Per Second : 50.0
Frames Per Second : 38.46153846153846
Frames Per Second : 38.46153846153846
Frames Per Second : 41.666666666666664


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
# [ PERTANYAAN ]
# TODO: Apa perbedaan "image classification" dan "object detection"?

Perbedaan image classification dan object detection adalah jika image classification itu berfungsi untuk mengklasifikasikan seluruh gambar ke dalam satu atau beberapa kategori tertentu. Contohnya adalah jika dalam sebuah gambar berisi mobil, image classification akan memberi label "mobil" untuk gambar tersebut. Model tidak peduli apakah gambar tersebut berisi satu atau beberapa mobil, atau lokasi dari objek tersebut di dalam gambar.

Sedangkan object detection itu berfungsi mengidentifikasi dan memberi label objek-objek tertentu dalam gambar serta menunjukkan lokasi (bounding box) dari objek-objek tersebut. Contohnya jika sebuah gambar berisi mobil dan truck, object detection akan mengidentifikasi kedua objek tersebut, memberi label "mobil" dan "truck", serta menandai lokasi kedua objek dengan kotak pembatas (bounding box).

[ ANSWER HERE ]

In [ ]:
# [ PERTANYAAN ]
# TODO: Di video mana YOLOv5 memiliki akurasi deteksi terburuk? Mengapa?

Video yang memiliki akurasi paling buruk adalah video tata surya pada link https://www.youtube.com/watch?v=g2KmtA97HxY. Hal ini dikarenakan model YOLO yang dipakai adalah model pretrained. Melalui dokumentasi yang diberikan dari ultralytics YOLOv5, model pretrained menggunakan dataset COCO (Common Objects in Context). Dataset COCO ini berisi objek-objek umum dari kehidupan sehari-hari seperti manusia, hewan, mobil, dll. Oleh karena itu objek luar angkasa / tata surya tidak dikenali oleh model ini karena pada dataset mereka itu tidak ada.

[ ANSWER HERE ]

### Scoring
Total `#TODO` = 4
<br>Checklist:

- [ ] Panggil model ultralytics/yolov5
- [ ] Isi parameter dengan URL YouTube yang tersedia
- [ ] [ PERTANYAAN ] Apa perbedaan "image classification" dan "object detection"?
- [ ] [ PERTANYAAN ] Di video mana YOLOv5 memiliki akurasi deteksi terburuk? Mengapa?

### Additional readings
* N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.